# Standalone Prediction

In [121]:
# Imports
import pandas as pd

## Import des Modells

In [122]:
from joblib import load
reg = load('gradientboosted2.model')

## Import der Daten
Hier bitte entweder über den Input-Dialog, oder direkt, den Dateinamen eingeben.

In [123]:
filename = input("Bitte Dateiname der CSV-Datei eingeben:")

In [124]:
dataset = pd.read_csv(filename, delimiter=';', encoding = 'cp852')
dataset

,Grundstück in qm,Grundstücksform,Steigung,Bezirk,Zone,Lage,Typ,Zustand,Gebaut,Renoviert,...,Schlafzimmer,Küchen,Küchenqualitt,Rume,Garage Typ,Garagenkapazitt,Pool,Verkaufsmonat,Verkaufsjahr,Preis
0,1093,Reg,Nein,Grand Park,RL,Norm,1Fam,7,2040,2130,...,2,1,3,6,Freistehend,1,NaN,5,2137,112000
1,1228,IR1,Nein,North East,RL,Norm,1Fam,5,2134,2134,...,3,1,4,9,Eingebaut,3,NaN,2,2136,279000
2,778,Reg,Nein,Somerset,RL,Norm,1Fam,7,2071,2103,...,2,1,4,6,2Typen,3,NaN,6,2138,152400


## Vorbereitung der Daten
1. Entfernung unnötiger Spalten
2. One-Hot Encoding
3. Entfernen encodeter ursprünglicher Spalten
4. Auffüllen mit fehlenden One-Hot Encoding Spalten
5. Sortieren der Spalten um Modell zu entsprechen

In [125]:
df_keinunnoetig = dataset.drop(['Verkaufsjahr', 'Verkaufsmonat', 'Erster Stock in qm', 'Zweiter Stock in qm', 'Pool', 'Küchen', 'Klimaanlage', 'Heizung', 'Preis'], axis = 1)
encode_kategorien = ['Bezirk', 'Lage', 'Heizungsqualitt', 'Typ', 'Steigung', 'Grundstücksform', 'Garage Typ', 'Zone']
encoded_kategorieeigenschaften = pd.get_dummies(dataset[encode_kategorien].fillna('Keine'))
df_encoded = df_keinunnoetig.drop(encode_kategorien, axis = 1).join(encoded_kategorieeigenschaften)

needed_columns = ['Grundstück in qm', 'Zustand', 'Gebaut', 'Renoviert', 'Zustand Fassade',
       'Kellerflche in qm', 'Wohnflche in qm', 'Schlafzimmer', 'Küchenqualitt',
       'Rume', 'Garagenkapazitt', 'Bezirk_Burnley', 'Bezirk_ChinaTown',
       'Bezirk_City Hall', 'Bezirk_Clear Creek', 'Bezirk_DiamondDistrict',
       'Bezirk_Dixon Side', 'Bezirk_East End', 'Bezirk_Fashion District',
       'Bezirk_Grand Park', 'Bezirk_Miller', 'Bezirk_New Gotham',
       'Bezirk_North East', 'Bezirk_North West', 'Bezirk_Novick District',
       'Bezirk_Old Gotham', 'Bezirk_Paris Island', 'Bezirk_Robinson Park',
       'Bezirk_Somerset', 'Bezirk_Somerset West', 'Bezirk_Somerst',
       'Bezirk_The Bowery', 'Bezirk_Tricorner', 'Bezirk_University',
       'Bezirk_Upper West', 'Bezirk_Water District', 'Lage_Bahn', 'Lage_Norm',
       'Lage_Pos', 'Lage_Str', 'Heizungsqualitt_Ex', 'Heizungsqualitt_Gut',
       'Heizungsqualitt_Schl', 'Heizungsqualitt_Ty', 'Heizungsqualitt_Ud',
       'Typ_1Fam', 'Typ_2Fam', 'Typ_2FamAn', 'Typ_REH', 'Typ_RMH',
       'Steigung_Ja', 'Steigung_Nein', 'Grundstücksform_IR1',
       'Grundstücksform_IR2', 'Grundstücksform_IR3', 'Grundstücksform_Reg',
       'Garage Typ_2Typen', 'Garage Typ_Anbau', 'Garage Typ_CarPort',
       'Garage Typ_Eingebaut', 'Garage Typ_Freistehend', 'Garage Typ_Keine',
       'Garage Typ_TG', 'Zone_RH', 'Zone_RL', 'Zone_RM',
       'Bezirk_Finger River']

print("Fehlende Eigenschaften:",set(needed_columns).difference(set(df_encoded.columns)))
print("Eigenschaften werden jetzt angefügt..")

for column in needed_columns:
    if column not in df_encoded:
        df_encoded[column] = 0

df_encoded = df_encoded.reindex(columns = needed_columns)

Fehlende Eigenschaften: {'Typ_2FamAn', 'Bezirk_Paris Island', 'Typ_REH', 'Garage Typ_TG', 'Lage_Bahn', 'Bezirk_Water District', 'Bezirk_New Gotham', 'Lage_Pos', 'Bezirk_The Bowery', 'Heizungsqualitt_Ud', 'Zone_RM', 'Bezirk_Dixon Side', 'Zone_RH', 'Bezirk_Finger River', 'Bezirk_Miller', 'Bezirk_Somerst', 'Bezirk_Old Gotham', 'Heizungsqualitt_Schl', 'Bezirk_Robinson Park', 'Bezirk_Burnley', 'Bezirk_City Hall', 'Garage Typ_Anbau', 'Garage Typ_Keine', 'Typ_2Fam', 'Grundstücksform_IR3', 'Bezirk_East End', 'Bezirk_Novick District', 'Bezirk_DiamondDistrict', 'Bezirk_North West', 'Bezirk_Tricorner', 'Typ_RMH', 'Bezirk_ChinaTown', 'Bezirk_Clear Creek', 'Bezirk_University', 'Steigung_Ja', 'Garage Typ_CarPort', 'Heizungsqualitt_Ty', 'Bezirk_Upper West', 'Bezirk_Fashion District', 'Lage_Str', 'Grundstücksform_IR2', 'Bezirk_Somerset West'}
Columns werden jetzt angefügt..


## Prediction

In [126]:
y_pred = reg.predict(df_encoded)

In [127]:
dataset['Vorhergesagter Preis'] = y_pred

In [128]:
dataset

,Grundstück in qm,Grundstücksform,Steigung,Bezirk,Zone,Lage,Typ,Zustand,Gebaut,Renoviert,...,Küchen,Küchenqualitt,Rume,Garage Typ,Garagenkapazitt,Pool,Verkaufsmonat,Verkaufsjahr,Preis,Vorhergesagter Preis
0,1093,Reg,Nein,Grand Park,RL,Norm,1Fam,7,2040,2130,...,1,3,6,Freistehend,1,NaN,5,2137,112000,116338.105034
1,1228,IR1,Nein,North East,RL,Norm,1Fam,5,2134,2134,...,1,4,9,Eingebaut,3,NaN,2,2136,279000,263534.864965
2,778,Reg,Nein,Somerset,RL,Norm,1Fam,7,2071,2103,...,1,4,6,2Typen,3,NaN,6,2138,152400,139496.105800


## Metriken berechnen
Folgend sollen die folgenden Metriken für den eingelesenen Datensatz berechnet werden:  
R2, MSE, RMSE, MAPE, MAX.

In [129]:
from sklearn.metrics import r2_score, mean_squared_error, max_error
import numpy as np

def mape (y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [130]:
y_true = dataset['Preis']
print("R^2:", r2_score(y_true, y_pred))
print("MSE:", mean_squared_error(y_true, y_pred))
print("RMSE:", mean_squared_error(y_true, y_pred)**.5)
print("MAPE:", mape(y_true, y_pred))
print("MAX:", max_error(y_true, y_pred))

R^2: 0.972040923930794
MSE: 141500014.1483296
RMSE: 11895.377848068954
MAPE: 5.961163132108747
MAX: 15465.135034807201
